#정보보호와 시스템보안
##AI악성코드탐지 프로젝트
> 팀이름
+ 이헌재(20163148)
+ 김동욱(20163090)
+ 장한영(20171696)




###구글드라이브 연동 및 데이터 압축해제

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!unzip /content/drive/My\ Drive/데이터.zip

###라이브러리 import

In [ ]:
import os
import glob
import json
import pprint

import numpy as np

from lightgbm import LGBMClassifier

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE

###학습시 사용 함수 정의

In [ ]:
SEED = 41

def read_label_csv(path):
    label_table = dict()
    with open(path, "r", encoding="cp949") as f:
        for line in f.readlines()[1:]:
            fname, label = line.strip().split(",")
            label_table[fname] = int(label)
    return label_table

def read_json(path):
    with open(path, "r") as f:
        return json.load(f)

def load_model(**kwargs):
    if kwargs["model"] == "rf":
        return RandomForestClassifier(random_state=kwargs["random_state"], n_jobs=4)
    elif kwargs["model"] == "dt":
        return DecisionTreeClassifier(random_state=kwargs["random_state"])
    elif kwargs["model"] == "lgb":
        return LGBMClassifier(random_state=kwargs["random_state"])
    elif kwargs["model"] == "svm":
        return SVC(random_state=kwargs["random_state"])
    elif kwargs["model"] == "lr":
        return LogisticRegression(random_state=kwargs["random_state"], n_jobs=-1)
    elif kwargs["model"] == "knn":
        return KNeighborsClassifier(n_jobs=-1)
    elif kwargs["model"] == "adaboost":
        return AdaBoostClassifier(random_state=kwargs["random_state"])
    elif kwargs["model"] == "mlp":
        return MLPClassifier(random_state=kwargs["random_state"])
    else:
        print("Unsupported Algorithm")
        return None
    

def train(X_train, y_train, model):
    '''
        머신러닝 모델을 선택하여 학습을 진행하는 함수
	
        :param X_train: 학습할 2차원 리스트 특징벡터
        :param y_train: 학습할 1차원 리스트 레이블 벡터
        :param model: 문자열, 선택할 머신러닝 알고리즘
        :return: 학습된 머신러닝 모델 객체
    '''
    clf = load_model(model=model, random_state=SEED)
    clf.fit(X_train, y_train)
    return clf


def evaluate(X_test, y_test, model):
    '''
        학습된 머신러닝 모델로 검증 데이터를 검증하는 함수
	
        :param X_test: 검증할 2차원 리스트 특징 벡터
        :param y_test: 검증할 1차원 리스트 레이블 벡터
        :param model: 학습된 머신러닝 모델 객체
    '''
    predict = model.predict(X_test)
    print("정확도", model.score(X_test, y_test))


### 레이블 테이블 로드

In [ ]:
label_table = read_label_csv("/content/학습데이터_정답.csv")
label_table2 = read_label_csv("/content/검증데이터_정답.csv")

### 특징 벡터 생성 예시
- PEMINER 정보는 모두 수치형 데이터이므로 특별히 가공을 하지 않고 사용 가능
- EMBER, PESTUDIO 정보는 가공해서 사용해야 할 특징들이 있음 (e.g. imports, exports 등의 문자열 정보를 가지는 데이터)
- 수치형 데이터가 아닌 데이터(범주형 데이터)를 어떻게 가공할 지가 관건 >> 인코딩 (e.g. 원핫인코딩, 레이블인코딩 등)

##PEMINER 특징 추출
---
###peminer 특징 벡터 제거 코드
- peminer 기존 전체 188개의 피처중 학습을 방해하는 특징 벡터를 제거하기 위함

- for문을 통해 피처를 한개씩 제거하여 정확도가 오르는 특징 벡터들을 확인
  - 결과 : [20,50,51,56,72,73,74,80,102,106,107,149,150,151,152,153,154,156,159,165,168]를 제거 하였을때 가장 높은 정확도가 나왔음 

```python
import os
# ax에서 하나씩 빼보면서 peminer을 돌려서 정확도가 올라가는걸 찾음
ax=[20,50,51,56,72,73,74,80,102,106,107,149,150,151,152,153,154,156,159,165,168]


am=0.9551
path_dir = f"/content/PEMINER/학습데이터"

path_dir2 = f"/content/PEMINER/검증데이터"
file_list = os.listdir(path_dir)

file_list2 = os.listdir(path_dir2)

for s in range(len(ax)):
  ass= ax[:]
  del ass[s]
  X, y = [], []
  X_test, y_test = [], []
  for i in range(len(file_list)):
    feature_vector = []
    for data in ["PEMINER"]:
        path = f"/content/{data}/학습데이터/{file_list[i]}"
        label = label_table[file_list[i].split('.')[0]]
        if data == "PEMINER": 
            feature_vector += PeminerParser(path).process_report(ass)
    X.append(feature_vector)
    y.append(label)
  for i in range(len(file_list2)):
    feature_vector = []
    for data in ["PEMINER"]:
        path = f"/content/{data}/검증데이터/{file_list2[i]}"
        label = label_table2[file_list2[i].split('.')[0]]
        if data == "PEMINER": 
            feature_vector += PeminerParser(path).process_report(ass)
    X_test.append(feature_vector)
    y_test.append(label)
 
  models = []
  for model in ["rf", "lgb"]:
      clf = train(X, y, model)
      models.append(clf)

  for model in models:
      evaluate(X_test, y_test, model)
  def ensemble_result(X, y, models):
    
    predicts = []
    for model in models:
        prob = [result for _, result in model.predict_proba(X)]
        predicts.append(prob)
    
    predict = np.mean(predicts, axis=0)
    predict = [1 if x >= 0.5 else 0 for x in predict]
        
    global am
    if accuracy_score(y, predict)>am:
     
      

      am=accuracy_score(y, predict)
      print("ax정확도",s, accuracy_score(y, predict))
  ensemble_result(X_test, y_test, models)
print(ax)
```




In [ ]:
class PeminerParser:
    def __init__(self, path):
        self.report = read_json(path)
        self.vector = []
    
    def process_report(self):

        a=[20,50,51,56,72,73,80,102,106,107,149,150,151,152,153,154,156,159,165,168]
        vector3 = [value for _, value in sorted(self.report.items(), key=lambda x: x[0])]

        for i in range(len(a)):
          del vector3[a[i]-i]

        self.vector = vector3

        return self.vector

### EMBER 특징 추출
---
- 기존에 있던 특징들을 수정한다.
 > 'general'과 'strings'에 있던 특징들을 하나씩 제거해보며 정확도가 증가하면 제거, 감소하면 다시 특징으로 추출한다.-> 결국 기존에 있던 특징 제거

- 새로운 특징을 찾아 vector에 하나씩 추가하여 정확도가 증가하면 추가, 감소하면 제거한다.
 - 증가한 특징
   > get_entropy_max(), get_bigentropy_len(), numimport(), get_byteentropy_max(), get_histogram_max(), get_datadirsize_max() ,numexport(), get_datadirnum(), get_sizeof_code(), get_byteennum()
 - 감소한 특징
   > get_general_file_info(), get_histogram_info(), get_string_info(), get_sectionsize_max(), get_sectionvsize_max(), get_byteentropy_info(), et_max_entropy(), get_string_max()

In [ ]:
class EmberParser:
    '''
        예제에서 사용하지 않은 특징도 사용하여 벡터화 할 것을 권장
    '''
    def __init__(self, path):
        self.report = read_json(path)
        self.vector = []
    
    def get_histogram_info(self):
        histogram = np.array(self.report["histogram"])
        total = histogram.sum()
        vector = histogram / total
        return vector.tolist()
    
    def get_string_info(self):
        strings = self.report["strings"]

        hist_divisor = float(strings['printables']) if strings['printables'] > 0 else 1.0
        vector = [
            strings['numstrings'], 
            strings['avlength'], 
            strings['printables'],
            strings['entropy'], 
            #strings['paths'], 
            #strings['urls'],
            strings['registry'], 
            strings['MZ']
        ]
        vector += (np.asarray(strings['printabledist']) / hist_divisor).tolist()
        return vector
    
    def get_general_file_info(self):
        general = self.report["general"]
        vector = [
            #general['size'], 
            general['vsize'], 
            general['has_debug'], 
            general['exports'], 
            general['imports'],
            general['has_relocations'], 
            general['has_resources'], 
            general['has_signature'], 
            general['has_tls'],
            #general['symbols']
        ]
        return vector
    # 특징 추가
    def get_byteentropy_info(self):
        byteentropy = np.array(self.report["byteentropy"])
        total = byteentropy.sum()
        vector = byteentropy / total
        return vector.tolist()
        
    def get_byteentropy_max(self):
        byteentropy = np.array(self.report["byteentropy"])
        maxi = np.max(byteentropy)
        vector = [maxi]
        return vector

    def get_histogram_max(self):
        histogram = np.array(self.report["histogram"])
        maxi = np.max(histogram)
        vector = [maxi]
        return vector

    def get_string_max(self):
        string_max = np.array(self.report["strings"]['printabledist'])
        maxi = np.max(string_max)
        vector = [maxi]
        return vector

    def get_datadirsize_max(self):
        data = np.array(self.report["datadirectories"])
        aaaa=[-1]
        for i in range(len(data)):
          aaaa.append(data[i]["size"])
        vector = [max(aaaa)]
        return vector

    def get_sectionsize_max(self):
        section = np.array(self.report["section"]["sections"])
        aaaa=[999999999]
        for i in range(len(section)):
          aaaa.append(section[i]["size"])
        vector = [min(aaaa)]
        return vector

    def get_sectionvsize_max(self):
        section = np.array(self.report["section"]["sections"])
        aaaa=[999999999]
        for i in range(len(section)):
          aaaa.append(section[i]["vsize"])
        vector = [min(aaaa)]
        return vector

    def get_entropy_max(self):
        section = np.array(self.report["section"]["sections"])
        aaaa=[999999999]
        for i in range(len(section)):
          aaaa.append(int(section[i]["entropy"]))
        vector = [max(aaaa)]
        return vector

    def get_bigentropy_len(self):
        section = np.array(self.report["section"]["sections"])
        aaaa=0
        for i in range(len(section)):
          if section[i]["entropy"]>=6:
            aaaa+=1 
        vector = [aaaa]
        return vector

    def numimport(self):
        import1 = self.report["imports"]
        try:
          vector = [len(import1)]
        except:
          vector =[-1] 
        return vector

    def numexport(self):
        export1 = self.report["exports"]
        try:
          vector = [len(export1)]
        except:
          vector =[-1] 
        return vector

    def get_datadirnum(self):
        data = np.array(self.report["datadirectories"])
        vector = [len(data)]
        return vector

    def get_sizeof_code(self):
        sizeofcode = np.array(self.report["header"]["optional"]["sizeof_code"])
  
        vector = [float(sizeofcode)]
        return vector

    def get_byteennum(self):
        data = np.array(self.report["byteentropy"])
        vector = [len(data)]
        return vector

    def process_report(self):
        vector = []
        #vector += self.get_general_file_info()
        #vector += self.get_histogram_info()
        #vector += self.get_string_info()    
        #vector += self.get_sectionsize_max()
        #vector += self.get_sectionvsize_max()
        #vector += self.get_byteentropy_info()
        #vector += self.get_max_entropy()
        #vector += self.get_string_max()     

        vector += self.get_entropy_max()
        vector += self.get_bigentropy_len()
        vector += self.numimport()       
        vector += self.get_byteentropy_max()
        vector += self.get_histogram_max()        
        vector += self.get_datadirsize_max()
        vector += self.numexport()
        vector += self.get_datadirnum()
        vector += self.get_sizeof_code()
        vector += self.get_byteennum()
        '''
            특징 추가
        '''
        return vector

### PESTUDIO 특징 추출
---
- 새로운 특징을 찾아 vector에 하나씩 추가하여 정확도가 증가하면 추가, 감소하면 제거한다.
 - 증가한 특징
   > get_entropy_max(), get_bigentropy_len(), numimport(), get_byteentropy_max(), get_histogram_max(), get_datadirsize_max(), numexport(), get_datadirnum(), get_sizeof_code(), get_byteennum()
 - 감소한 특징
   > get_general_file_info(), get_histogram_info(), get_string_info(), get_sectionsize_max(), get_sectionvsize_max(), get_byteentropy_info(), get_max_entropy(), get_string_max()

In [ ]:
class PestudioParser:
    '''
        사용할 특징을 선택하여 벡터화 할 것을 권장
    '''
    def get_indicators_info(self):
        count = 0;
        try:
          image = self.report["image"]["indicators"]["indicator"]
          for i in range(0, len(image)):
            if(image[i]["@severity"] == "1" or image[i]["@severity"] == "2"):
              count+=1
        except:
          count = -1
        vector = [count]
        return vector

    def numsection(self):
        try:
          section = self.report["image"]["sections"]["section"]
          vector = [(len(section))]
        except:
          vector =[-1] 
        return vector

    def blacklist(self):
        try:
          section = self.report["image"]["sections"]["section"]
          count = 0
          for i in range(len(section)):
            if section[i]["@blacklisted"] == "x":
                count += 1
        except:
          count =-1
        vector = [count]
        return vector

    def import_blacklist(self):
        try:
          imports = self.report["image"]["imports"]["import"]
          count = 0
          for i in range(len(imports)):
              if imports[i]["@blacklist"] == "x":
                count += 1
        except:
          count = -1
        vector = [count]
        return vector

    def libraries_blacklist(self):
        try:
          libraries = self.report["image"]["libraries"]["library"]
          count = 0
          for i in range(len(libraries)):
              if libraries[i]["@blacklist"] == "x":
                count += 1
        except:
          count = -1
        vector = [count]
        return vector

    def overview_entropy(self):
        try:
          entropy = self.report["image"]["overview"]["entropy"]
          vector = [float(entropy)]
        except:
          vector = [-1]
        return vector

    def size(self):
        try:
          vec=[]
          instance = self.report["image"]["resources"]["instance"]
          for i in range(len(instance)):
            vec.append(int(instance[i]['@size']))
          vector = [max(vec)]
        except:
          vector = [-1]
        return vector     

    def entropy(self):
        try:
          vec=[]
          instance = self.report["image"]["resources"]["instance"]
          for i in range(len(instance)):
            vec.append(int(instance[i]['@entropy']))
          vector = [max(vec)]
        except:
          vector = [-1]
        return vector   

    def string_size_max(self):
        try:
          vec=[]
          string = self.report["image"]["strings"]["string"]
          for i in range(len(string)):
            vec.append(int(string[i]['@size']))
          vector = [max(vec)]
        except:
          vector = [-1]
        return vector  

    def get_debug(self):
        try:
          image = self.report["debug"]
          if image['debug'] == 'n/a' :
            vector = [0]
          else :
            vector = [1]  
        except :
            vector = [-1]      
        return vector

    def get_string_leng(self):
        try:
          image = self.report["image"]
          vector = [ len(image['strings']['ascii']['string']) ]
        except:
          vector = [-1]
        return vector

    def get_tls_callbacks(self):
        try:
          image = self.report["image"]
          if image['tls-callbacks'] == 'n/a' :
            vector = [0]
          else :
            vector = [1]
        except :
          vector = [-1]
        return vector

    def get_certificate(self):
        try:
          image = self.report["image"]
          if image['certificate'] == 'n/a' :
            vector = [0]
          else :
            vector = [1]
        except :
          vector = [-1]
        return vector

    def get_overlay(self):
        try : 
          image = self.report["image"]
          if image['overlay'] == 'n/a' :
            vector = [0]
          else :
            vector = [1]      
        except:
          vector = [-1]
        return vector   

    def __init__(self, path):
        try:
          self.report = read_json(path)
        except:
          self.report = None
        self.vector = []

    def process_report(self):
        vector = []

        #vector += self.get_debug()
        #vector += self.get_string_leng()
        #vector += self.get_tls_callbacks()
        #vector += self.get_certificate()
        #vector += self.get_overlay()
        #vector += self.size()

        vector += self.string_size_max()
        vector += self.entropy()
        vector += self.get_indicators_info()
        vector += self.numsection()
        vector += self.blacklist()
        vector += self.import_blacklist()
        vector += self.libraries_blacklist()
        vector += self.overview_entropy()

        return vector

#### 특징 추가, 삭제, 변경에 따른 정확도 기록
https://docs.google.com/document/d/1K17iDGk5jYcMP_y0-LQnURWK7TYkvGIzRR5FNPrpWhw/edit?usp=sharing

### 학습데이터 구성
 - 데이터의 특징 벡터 모음(2차원 리스트) : X
 - 데이터의 레이블 모음(1차원 리스트) : y
 - 이부분에서 모든 파일을 읽어와야 한다 엠버와 피마이너 파일의 학습데이터 안에 들어있는
 - **주의사항**
   1. 특징 벡터 구성은 2차원이여야 한다.
   2. 각 벡터는 1차원 리스트, 벡터 크기는 모두 같아야함

In [ ]:
X, y = [], []

path_dir = f"/content/PEMINER/학습데이터"

file_list = os.listdir(path_dir)

for i in range(len(file_list)):
    feature_vector = []
    for data in ["PEMINER","EMBER","PESTUDIO"]:
        path = f"/content/{data}/학습데이터/{file_list[i]}"
        label = label_table[file_list[i].split('.')[0]]
        if data == "PEMINER": 
            feature_vector += PeminerParser(path).process_report()
        elif data == "EMBER":
            feature_vector += EmberParser(path).process_report()
        elif data == "PESTUDIO":
            feature_vector += PestudioParser(path).process_report()
            
    X.append(feature_vector)
    y.append(label)

print(np.asarray(X).shape)

(20000, 186)


###검증데이터 구성

 - 데이터의 특징 벡터 모음(2차원 리스트) : X_test
 - 데이터의 레이블 모음(1차원 리스트) : y_test

In [ ]:
X_test, y_test = [], []

path_dir = f"/content/PEMINER/검증데이터"

file_list = os.listdir(path_dir)

for i in range(len(file_list)):
    feature_vector = []
    for data in ["PEMINER","EMBER","PESTUDIO"]:
        path = f"/content/{data}/검증데이터/{file_list[i]}"
        label = label_table2[file_list[i].split('.')[0]]
        if data == "PEMINER": 
            feature_vector += PeminerParser(path).process_report()
        elif data == "EMBER":
            feature_vector += EmberParser(path).process_report()
        elif data == "PESTUDIO":
            feature_vector += PestudioParser(path).process_report()
    
    X_test.append(feature_vector)
    y_test.append(label)
    
print(np.asarray(X_test).shape)

(10000, 186)


###학습데이터(2만개)+검증데이터(1만개) 모두 학습

In [ ]:
X, y = [], []

path_dir = f"/content/PEMINER/학습데이터"

file_list = os.listdir(path_dir)

for i in range(len(file_list)):
    feature_vector = []
    for data in ["PEMINER","EMBER","PESTUDIO"]:
        path = f"/content/{data}/학습데이터/{file_list[i]}"
        label = label_table[file_list[i].split('.')[0]]
        if data == "PEMINER": 
            feature_vector += PeminerParser(path).process_report()
        elif data == "EMBER":
            feature_vector += EmberParser(path).process_report()
        elif data == "PESTUDIO":
            feature_vector += PestudioParser(path).process_report()
            
    X.append(feature_vector)
    y.append(label)

print(np.asarray(X).shape)

path_dir = f"/content/PEMINER/검증데이터"

file_list = os.listdir(path_dir)

for i in range(len(file_list)):
    feature_vector = []
    for data in ["PEMINER","EMBER","PESTUDIO"]:
        path = f"/content/{data}/검증데이터/{file_list[i]}"
        label = label_table2[file_list[i].split('.')[0]]
        if data == "PEMINER": 
            feature_vector += PeminerParser(path).process_report()
        elif data == "EMBER":
            feature_vector += EmberParser(path).process_report()
        elif data == "PESTUDIO":
            feature_vector += PestudioParser(path).process_report()
    
    X.append(feature_vector)
    y.append(label)
    
print(np.asarray(X).shape, np.asarray(y).shape)

(20000, 186)
(30000, 186) (30000,)


In [ ]:
models = []
for model in ["rf", "lgb"]:
    clf = train(X, y, model)
    models.append(clf)

### 학습 및 검증(최종 예측 결과에는 검증이 쓰이지 않음)
---
알고리즘의 종류와 개수를 다양하게 앙상블 한다.
  - algo2는 서로 다른 2개의 알고리즘을 갖는 2차원 리스트다.
  - 마찬가지로 algo3, algo4 .... 점차 개수를 늘려 다양하게 넣어 앙상블 한다.
  - 이 중 최고의 정확도를 가지는 알고리즘 algoMaxName과 정확도 accuracyMax를 취한다.
    - *** Max accuracy:  ['rf', 'lgb'] 0.9607 ***

```python
algorithm = ["rf", "lgb","dt","lr", 'svm', 'knn', 'adaboost', 'mlp']
algo2 = []
for i in range(len(algorithm))):
  for j in range(i+1,len(algorithm)): 
    algo2.append( [ algorithm[i], algorithm[j] ] )

accuracyMax = 0
algoMaxName = []

for algotmp in algo2 :
  models = []
  try:
    for model in algotmp:
      clf = train(X, y, model)
      models.append(clf)
    
    accu = ensemble_result_test(X_test, y_test, models)   
    
    if accu > accuracyMax :
        accuracyMax = accu
        algoMaxName = algotmp
        print(algotmp, accu)
    elif accu == -1 :
        print(algotmp, "함수 에러")  
    else :
        print(algotmp, accu)
  except :
    print(algotmp, "알고리즘 에러")

print("Max accuracy: ", algoMaxName, accuracyMax)

def ensemble_result_test(X, y, models):
    try: 
      predicts = []
      for model in models:
          prob = [result for _, result in model.predict_proba(X)]
          predicts.append(prob)
    
      predict = np.mean(predicts, axis=0)
      predict = [1 if x >= 0.5 else 0 for x in predict]
      return accuracy_score(y, predict)

    except :
      return -1
```

In [ ]:
# 학습 피마이너랑 엠버를 학습시켜서 학습된 특징벡터들을 models에 넣는다 
models = []
for model in ["rf", "lgb"]:
    clf = train(X, y, model)
    models.append(clf)

# 검증 피마이너랑 엠버를 각각 검증 
# 실제 검증 시에는 제공한 검증데이터를 검증에 사용해야 함
for model in models:
    evaluate(X_test, y_test, model)

def ensemble_result(X, y, models):
    '''
        학습된 모델들의 결과를 앙상블하는 함수
   
        :param X: 검증할 2차원 리스트 특징 벡터
        :param y: 검증할 1차원 리스트 레이블 벡터
        :param models: 1개 이상의 학습된 머신러닝 모델 객체를 가지는 1차원 리스트
    '''
    predicts = []
    for model in models:
        prob = [result for _, result in model.predict_proba(X)]
        predicts.append(prob)
    
    predict = np.mean(predicts, axis=0)
    predict = [1 if x >= 0.5 else 0 for x in predict]
        
    print("앙상블 후 정확도", accuracy_score(y, predict))

ensemble_result(X_test, y_test, models)


정확도 0.9557
정확도 0.9538
앙상블 후 정확도 0.9607


###테스트 데이터 구성

In [ ]:
final_test = []
labeltable_test = []

path_dir = f"/content/PEMINER/테스트데이터"

file_list = os.listdir(path_dir)

for i in range(len(file_list)):
    feature_vector = []
    for data in ["PEMINER","EMBER","PESTUDIO"]:
        path = f"/content/{data}/테스트데이터/{file_list[i]}"
        label = file_list[i].split('.')[0]
        if data == "PEMINER": 
            feature_vector += PeminerParser(path).process_report()
        elif data == "EMBER":
            feature_vector += EmberParser(path).process_report()
        elif data == "PESTUDIO":
            feature_vector += PestudioParser(path).process_report()
    final_test.append(feature_vector)
    labeltable_test.append(label)

print(np.asarray(final_test).shape)

(10000, 186)


### 테스트 데이터 학습 및 앙상블
테스트 데이터는 결과가 없기 때문에 y 데이터가 없다.

In [ ]:
def ensemble_data(X, models):
    '''
        학습된 모델들의 결과를 앙상블하는 함수
          
        :param X: 검증할 2차원 리스트 특징 벡터
        :param models: 1개 이상의 학습된 머신러닝 모델 객체를 가지는 1차원 리스트
    '''

    predicts = []
    for model in models:
        prob = [result for _, result in model.predict_proba(X)]
        predicts.append(prob)
    
    predict = np.mean(predicts, axis=0)
    predict = [ 1 if x >= 0.5 else 0 for x in predict ]
        
    return predict
  
result = ensemble_data(final_test, models)


###테스트 데이터 예측.csv 파일 생성

In [ ]:
import csv

predict_test = []

for i in range(len(labeltable_test)):
    predict_test.append([labeltable_test[i], result[i]])

path = "/content/drive/MyDrive/predict.csv"

with open(path, 'w', newline='') as f:
    w = csv.writer(f)
    w.writerow(['file', 'predict'])

    for i in predict_test :
        w.writerow(i)
        
    f.close()